In [52]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# 기본 라이브러리
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 모델 라이브러리
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, Lasso
import xgboost as xgb

# 평가 및 시각화
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor

# SHAP (선택사항)
try:
    import shap
    SHAP_AVAILABLE = True
except ImportError:
    print("SHAP 라이브러리가 설치되지 않았습니다. pip install shap 실행하세요.")
    SHAP_AVAILABLE = False

# 한글 폰트 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

In [30]:
data = pd.read_csv('df_merged.csv')

In [31]:
# 데이터 구조 확인
data.shape

(405, 26)

In [32]:
# 데이터 타입 
data.dtypes

month                     int64
CO2ppm                  float64
Temp                    float64
Humid                   float64
VPD                     float64
Chl_a                   float64
Chl_b                   float64
TChl                    float64
Car                     float64
Chl_a_b                 float64
TCh-Car                 float64
ABS-RC                  float64
Dio-RC                  float64
Tro-RC                  float64
Eto-RC                  float64
PI_abs                  float64
DF_abs                  float64
SFI_abs                 float64
Fv-Fm                   float64
Leaf_ExtractionYield    float64
Root_ExtractionYield    float64
Leaf_TPC                float64
Root_TPC                float64
Leaf_TFC                float64
Root_TFC                float64
scenario                 object
dtype: object

In [33]:
data

,month,CO2ppm,Temp,Humid,VPD,Chl_a,Chl_b,TChl,Car,Chl_a_b,...,DF_abs,SFI_abs,Fv-Fm,Leaf_ExtractionYield,Root_ExtractionYield,Leaf_TPC,Root_TPC,Leaf_TFC,Root_TFC,scenario
0,5,381.681033,16.918639,83.130786,1.532512,8.79,2.22,11.00,2.97,3.97,...,0.328,0.215,0.830,19.00,18.90,7.476,6.270,5.217,0.861,SSP1
1,5,374.463441,16.922124,83.096722,1.532868,8.99,2.56,11.55,3.09,3.52,...,0.287,0.199,0.826,20.10,19.60,7.369,6.396,5.257,0.836,SSP1
2,5,371.850683,16.930256,82.488003,1.534584,9.66,2.44,12.10,3.11,3.96,...,0.384,0.229,0.828,20.70,20.40,7.369,6.396,5.242,0.841,SSP1
3,5,400.475202,16.921511,82.081632,1.534512,9.33,2.45,11.79,3.13,3.80,...,0.503,0.282,0.839,19.00,18.90,7.476,6.270,5.217,0.861,SSP1
4,5,381.360788,16.921323,83.888666,1.531475,10.53,2.58,13.11,3.37,4.08,...,0.304,0.203,0.832,20.10,19.60,7.369,6.396,5.257,0.836,SSP1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,9,1208.463000,25.901000,60.192000,2.560000,2.64,0.52,3.15,1.07,5.09,...,-0.626,0.051,0.722,7.10,14.60,7.743,5.277,1.455,0.513,SSP5
401,9,1211.911000,25.896000,60.254000,2.559000,4.74,1.12,5.86,1.53,4.22,...,-0.829,0.046,0.588,7.05,14.55,7.760,5.245,1.450,0.507,SSP5
402,9,1206.015000,25.923000,59.125000,2.565000,2.52,0.08,2.60,1.38,31.49,...,-0.879,0.037,0.643,7.00,14.50,7.814,5.324,1.460,0.518,SSP5
403,9,1225.166000,25.890000,55.446000,2.571000,2.63,0.39,3.02,1.17,6.73,...,-0.478,0.067,0.727,7.10,14.60,7.743,5.277,1.455,0.513,SSP5


In [36]:
def hybrid_outlier_treatment(df):
    # 그룹 1: 이상치 보존 (극한 기후 조건 반영 필요)
    preserve_vars = ['Temp', 'VPD', 'CO2ppm']  # 기후 관련 변수
    
    # 그룹 2: Winsorization (11-17% 이상치)
    winsorize_vars = ['Fv-Fm', 'DF_abs', 'Leaf_TPC', 'Root_TPC', 'Humid']
    
    # 그룹 3: 이상치 플래깅 (5% 미만 이상치)
    # 문자열형 변수 제외
    num_cols = df.select_dtypes(include='number').columns.tolist()
    flag_vars = [col for col in num_cols if col not in preserve_vars + winsorize_vars]
    
    # Winsorization 적용 (95% 백분위수 사용)
    for col in winsorize_vars:
        lower = df[col].quantile(0.025)
        upper = df[col].quantile(0.975)
        df[col] = df[col].clip(lower=lower, upper=upper)
    
    # 이상치 플래깅
    outlier_flags = pd.DataFrame()
    for col in flag_vars:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        outlier_flags[f'{col}_outlier'] = ((df[col] < Q1 - 1.5*IQR) | 
                                          (df[col] > Q3 + 1.5*IQR)).astype(int)
    
    return pd.concat([df, outlier_flags], axis=1)

In [39]:
# 이상치 처리
df_processed = hybrid_outlier_treatment(data)
df_processed

,month,CO2ppm,Temp,Humid,VPD,Chl_a,Chl_b,TChl,Car,Chl_a_b,...,ABS-RC_outlier,Dio-RC_outlier,Tro-RC_outlier,Eto-RC_outlier,PI_abs_outlier,SFI_abs_outlier,Leaf_ExtractionYield_outlier,Root_ExtractionYield_outlier,Leaf_TFC_outlier,Root_TFC_outlier
0,5,381.681033,16.918639,81.596870,1.532512,8.79,2.22,11.00,2.97,3.97,...,0,0,0,0,0,0,0,1,0,0
1,5,374.463441,16.922124,81.596870,1.532868,8.99,2.56,11.55,3.09,3.52,...,0,0,0,0,0,0,0,1,0,0
2,5,371.850683,16.930256,81.596870,1.534584,9.66,2.44,12.10,3.11,3.96,...,0,0,0,0,0,0,0,1,0,0
3,5,400.475202,16.921511,81.596870,1.534512,9.33,2.45,11.79,3.13,3.80,...,0,0,0,0,0,0,0,1,0,0
4,5,381.360788,16.921323,81.596870,1.531475,10.53,2.58,13.11,3.37,4.08,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,9,1208.463000,25.901000,60.192000,2.560000,2.64,0.52,3.15,1.07,5.09,...,0,1,0,0,0,0,0,0,0,0
401,9,1211.911000,25.896000,60.254000,2.559000,4.74,1.12,5.86,1.53,4.22,...,0,1,0,0,0,0,0,0,0,0
402,9,1206.015000,25.923000,59.384597,2.565000,2.52,0.08,2.60,1.38,31.49,...,0,1,0,0,0,0,0,0,0,0
403,9,1225.166000,25.890000,59.384597,2.571000,2.63,0.39,3.02,1.17,6.73,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
class ClimateChangePredictionPipeline:
    """천궁 기후변화 시나리오 예측 파이프라인"""
    
    def __init__(self, data, target_columns=['Leaf_TPC', 'Root_TPC', 'Leaf_TFC', 'Root_TFC']):
        self.data = data.copy()
        self.target_columns = target_columns
        self.processed_data = {}
        self.models = {}
        self.results = {}
    
    def analyze_outliers(self, threshold=1.5):
        """이상치 분석 및 시각화"""
        print("="*60)
        print("이상치 분석 시작")
        print("="*60)
        
        numeric_cols = self.data.select_dtypes(include=[np.number]).columns
        outlier_info = {}
        
        for col in numeric_cols:
            Q1 = self.data[col].quantile(0.25)
            Q3 = self.data[col].quantile(0.75)
            IQR = Q3 - Q1
            
            lower_bound = Q1 - threshold * IQR
            upper_bound = Q3 + threshold * IQR
            
            outliers = self.data[(self.data[col] < lower_bound) | 
                                 (self.data[col] > upper_bound)]
            
            outlier_ratio = len(outliers) / len(self.data) * 100
            
            if outlier_ratio > 0:
                outlier_info[col] = {
                    'count': len(outliers),
                    'ratio': outlier_ratio,
                    'lower_bound': lower_bound,
                    'upper_bound': upper_bound
                }
        
        print("\n이상치가 있는 변수:")
        for col, info in sorted(outlier_info.items(), key=lambda x: x[1]['ratio'], reverse=True):
            print(f"{col}: {info['count']}개 ({info['ratio']:.2f}%)")
        
        return outlier_info
    
    def hybrid_outlier_treatment(self, df=None):
        """하이브리드 이상치 처리"""
        if df is None:
            df = self.data.copy()
        else:
            df = df.copy()
        
        print("\n하이브리드 이상치 처리 시작...")
        
        # 그룹 1: 이상치 보존 (극한 기후 조건)
        preserve_vars = ['Temp', 'VPD', 'CO2ppm'] if 'Temp' in df.columns else []
        
        # 그룹 2: Winsorization (11-17% 이상치)
        high_outlier_vars = ['Fv-Fm', 'DF_abs', 'Leaf_TPC', 'Root_TPC', 'Humid']
        winsorize_vars = [col for col in high_outlier_vars if col in df.columns]
        
        # 그룹 3: 이상치 플래깅
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        flag_vars = [col for col in numeric_cols 
                    if col not in preserve_vars + winsorize_vars + self.target_columns]
        
        # Winsorization 적용
        for col in winsorize_vars:
            if col not in self.target_columns:
                lower = df[col].quantile(0.025)
                upper = df[col].quantile(0.975)
                original_outliers = ((df[col] < lower) | (df[col] > upper)).sum()
                df[col] = df[col].clip(lower=lower, upper=upper)
                print(f"  {col}: Winsorization 적용 ({original_outliers}개 값 조정)")
        
        # 이상치 플래깅
        outlier_flags = pd.DataFrame(index=df.index)
        for col in flag_vars:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            outlier_flags[f'{col}_outlier'] = ((df[col] < Q1 - 1.5*IQR) | 
                                               (df[col] > Q3 + 1.5*IQR)).astype(int)
        
        df = pd.concat([df, outlier_flags], axis=1)
        print(f"  {len(outlier_flags.columns)}개 이상치 플래그 변수 생성")
        
        return df


In [ ]:
calculate_vif(df_processed)


전체 변수 VIF 값:
            Variable                VIF
              ABS-RC            2469526
              Tro-RC            1026835
              Dio-RC             416804
                TChl             317978
               Chl_a             204555
               Chl_b 18568.993505598224
              DF_abs 1909.3863061905413
                Temp  653.7007446326255
                 VPD   547.535293842733
             SFI_abs 491.83433463916174
              Eto-RC 167.68661605799474
              PI_abs 150.01207672576825
               Fv-Fm  143.6276307330615
                 Car  97.44406780395828
               month  74.37804489698502
            Root_TFC  33.79478505338218
             TCh-Car 30.095664970450734
            Root_TPC 21.157225144512992
            Leaf_TFC  13.98265956642094
Leaf_ExtractionYield 13.904128498024331
Root_ExtractionYield 10.668201936212956
              CO2ppm 10.328936552052017
               Humid  6.294325862141139
            Leaf_TPC 4.764

,Variable,VIF,VIF_formatted
11,ABS-RC,2.469526e+06,2469526
13,Tro-RC,1.026835e+06,1026835
12,Dio-RC,4.168041e+05,416804
7,TChl,3.179783e+05,317978
5,Chl_a,2.045552e+05,204555
6,Chl_b,1.856899e+04,18568.993505598224
16,DF_abs,1.909386e+03,1909.3863061905413
2,Temp,6.537007e+02,653.7007446326255
4,VPD,5.475353e+02,547.535293842733
17,SFI_abs,4.918343e+02,491.83433463916174


In [45]:
# 1. 변수 그룹화 및 대표 변수 선택
variable_groups = {
    '엽록소': ['TChl', 'Chl_a', 'Chl_b', 'Chl_a_b', 'Car', 'TCh-Car'],
    '형광': ['Fv-Fm', 'PI_abs', 'SFI_abs', 'DF_abs'],
    'RC반응': ['ABS-RC', 'Tro-RC', 'Dio-RC', 'Eto-RC'],
    '기후': ['Temp', 'Humid', 'VPD', 'CO2ppm'],
    '페놀': ['Leaf_TPC', 'Root_TPC'],
    '플라보노이드': ['Leaf_TFC', 'Root_TFC'],
    '추출수율': ['Leaf_ExtractionYield', 'Root_ExtractionYield']
}

# 2. 그룹별 처리 전략
def handle_multicollinearity(df, groups):
    processed_features = []
    
    for group_name, vars in groups.items():
        if group_name in ['페놀', '플라보노이드']:  # 타겟 관련 변수는 보존
            processed_features.extend(vars)
        elif group_name == '엽록소':
            # PCA로 2개 주성분 추출
            pca = PCA(n_components=2)
            pca_result = pca.fit_transform(df[vars])
            processed_features.extend([f'{group_name}_PC1', f'{group_name}_PC2'])
        elif group_name == 'RC반응':
            # 비율 변수 생성
            df['RC_efficiency'] = df['Tro-RC'] / df['ABS-RC']
            processed_features.append('RC_efficiency')
        else:
            # 상관관계 기반 대표 변수 선택
            corr_with_target = df[vars].corrwith(df['Root_TPC']).abs()
            best_var = corr_with_target.idxmax()
            processed_features.append(best_var)
    
    return processed_features

In [ ]:
df_result = handle_multicollinearity(df_processed, variable_groups)

['엽록소_PC1',
 '엽록소_PC2',
 'SFI_abs',
 'RC_efficiency',
 'Temp',
 'Leaf_TPC',
 'Root_TPC',
 'Leaf_TFC',
 'Root_TFC',
 'Root_ExtractionYield']

In [20]:
class ClimateChangePredictionPipeline:
    def __init__(self):
        self.models = {
            'rf': RandomForestRegressor(n_estimators=200, max_depth=10),
            'xgb': XGBRegressor(n_estimators=200, learning_rate=0.05),
            'ridge': Ridge(alpha=1.0),
            'lasso': Lasso(alpha=0.1)
        }
    
    def preprocess(self, df, scenario):
        # 1. 시나리오별 이상치 처리
        df = self.hybrid_outlier_treatment(df, scenario)
        
        # 2. 모델별 다중공산성 처리
        X_original = df.drop(columns=['Leaf_TPC', 'Root_TPC', 
                                      'Leaf_TFC', 'Root_TFC'])
        X_pca = self.apply_selective_pca(X_original)
        
        return {
            'tree_based': X_original,  # RF, XGBoost용
            'linear': X_pca  # Ridge, Lasso용
        }
    
    def evaluate(self, y_true, y_pred):
        return {
            'MAE': mean_absolute_error(y_true, y_pred),
            'MSE': mean_squared_error(y_true, y_pred),
            'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
            'R2': r2_score(y_true, y_pred),
            'MAPE': np.mean(np.abs((y_true - y_pred) / y_true)) * 100
        }

In [21]:
def extract_key_factors(model, X, feature_names):
    # 1. Permutation Importance
    perm_importance = permutation_importance(model, X, y, n_repeats=30)
    
    # 2. SHAP values (tree-based models)
    if isinstance(model, (RandomForestRegressor, XGBRegressor)):
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X)
        
    # 3. 통합 중요도 점수
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'permutation_importance': perm_importance.importances_mean,
        'model_importance': model.feature_importances_ if hasattr(model, 'feature_importances_') else None
    })
    
    return importance_df.nlargest(10, 'permutation_importance')